In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, walk
from os.path import join, basename, isdir
sys.path.append(str(Path(getcwd()).parent.parent))

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import codecs
import numpy as np
from glob import glob
from epiweeks import Week

In [ ]:
from bulletin import root, default_input, default_output, tables_path, root
default_output = join(default_output,'vacinacao')

from bulletin.systems import Notifica, CasosConfirmados, eSUS, Sivep
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import normalize_text, normalize_hash, date_hash

from datetime import datetime, date, timedelta

database_path = join(root,'database','leitos')

hoje = pd.to_datetime(date.today())
ontem = hoje - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
#LEITURA DO BANCO MAIS ATUAL
try:
    leitos_sus = pd.read_excel(join(default_input,'ocupacao_leitos','leitos_sus.xls'), dtype={'Data Nascimento':str, 'Data Internação':str,'Data Resultado Exame':str, 'Data Alta':str})
except:
    leitos_sus = pd.read_excel(join(default_input,'ocupacao_leitos','leitos_sus.xlsx'), dtype={'Data Nascimento':str, 'Data Internação':str,'Data Resultado Exame':str, 'Data Alta':str})

try:
    del(leitos_sus['Total de Linhas'])
except:
    pass

leitos_sus['Data Nascimento'] = leitos_sus['Data Nascimento'].str[:10]
leitos_sus['Data Nascimento'] = pd.to_datetime(leitos_sus['Data Nascimento'], format='%d/%m/%Y', errors='ignore')
leitos_sus['Data Nascimento'] = pd.to_datetime(leitos_sus['Data Nascimento'], format='%Y-%m-%d', errors='ignore')


leitos_sus['Data Internação'] = leitos_sus['Data Internação'].str[:10]
leitos_sus['Data Internação'] = pd.to_datetime(leitos_sus['Data Internação'], format='%d/%m/%Y', errors='ignore')
leitos_sus['Data Internação'] = pd.to_datetime(leitos_sus['Data Internação'], format='%Y-%m-%d', errors='ignore')


leitos_sus['Data Resultado Exame'] = leitos_sus['Data Resultado Exame'].str[:10]
leitos_sus['Data Resultado Exame'] = pd.to_datetime(leitos_sus['Data Resultado Exame'], format='%d/%m/%Y', errors='ignore')
leitos_sus['Data Resultado Exame'] = pd.to_datetime(leitos_sus['Data Resultado Exame'], format='%Y-%m-%d', errors='coerce')

leitos_sus['Data Alta'] = ''
leitos_sus['Data Alta'] = leitos_sus['Data Alta'].str[:10]
leitos_sus['Data Alta'] = pd.to_datetime(leitos_sus['Data Alta'], format='%Y-%m-%d', errors='coerce')

leitos_sus.shape

if 'Regional Residência' not in leitos_sus.columns:
    leitos_sus['Regional Residência'] = ''

leitos_sus = leitos_sus[['Cod Notificação COVID', 'Nome do Paciente', 'Data Nascimento', 'Idade', 'Leito Tipo', 'Data Internação', 'Data Alta', 'Cod Resultado Exame', 'Executante', 'Data Resultado Exame' ,'Município Residência', 'Regional Residência', 'Macrorregião Residencia']]

In [ ]:
leitos_sus.index = np.arange(len(leitos_sus))

In [ ]:
leitos_sus.dtypes

In [ ]:
leitos_sus = leitos_sus.drop(index=leitos_sus.loc[leitos_sus['Macrorregião Residencia']=='Macro Regional SP'].index)

In [ ]:
leitos_sus['HASH'] = leitos_sus['Cod Notificação COVID'].astype('str') + \
                     leitos_sus['Nome do Paciente'].apply(normalize_text) + \
                     leitos_sus['Leito Tipo'].apply(normalize_text)

In [ ]:
leitos_sus.loc[leitos_sus['Leito Tipo'].str.upper().str.contains('UTI'), 'TIPO DE LEITO'] = 'UTI'
leitos_sus.loc[~leitos_sus['Leito Tipo'].str.upper().str.contains('UTI'), 'TIPO DE LEITO'] = 'ENFERMARIA'

In [ ]:
leitos_sus = leitos_sus.drop_duplicates('HASH', keep='last')

In [ ]:
leitos_sus.to_pickle(join(database_path,'leitos_sus.pkl'))

In [ ]:
#LEITURA DO BANCO DE INTERNADOS E SALVAR OS DADOS DE HOJE
internamentos = pd.read_pickle(join(database_path,'internamentos.pkl'))

internamentos = pd.concat([internamentos, leitos_sus], ignore_index=True)
internamentos.index = np.arange(len(internamentos))

internamentos = internamentos.drop_duplicates('HASH', keep='last')

internamentos.to_pickle(join(database_path,'internamentos.pkl'))
internamentos.shape